In [ ]:
# https://github.com/mlr7/63-Years-of-Seismicity-in-California/blob/main/seismic_california_iris_1960to2023sept_v6.ipynb

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# fdsn client for connecting with web services, data pull
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

from datetime import datetime, timedelta

import math # rounding

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Draw box on map and get lat/lon out
# https://earthquake.usgs.gov/earthquakes/search/

# Enter a lat/lon box and viz on map
#https://dwtkns.com/pointplotter/
#https://earthquake.usgs.gov/earthquakes/search/

In [2]:
# All California lat/lon box:

longitude_xmax = -109.83
longitude_xmin = -127.672

latitude_ymax = 42.461
latitude_ymin = 28.938

# Useful notes:

#  longitude are verical lines on globe, parallel to prime meridian (longitude is x-axis tick marks!)
#  latitude are horiontal lines on globe, parallel to equator (latitude is y-axis tick marks!)

# Example:  latitude of California, USA is 36.778259, and the longitude is -119.417931
# Example:  Latitude of Pasadena, CA, USA is 34.156113, and the longitude is -118.131943. 

# https://www.yourdictionary.com/articles/latitude-vs-longitude

# Longitude is the points east and west of the prime meridian. 
# Longitude is x-axis tick marks!

# Latitude is the points north and south of the equator.
# latitude is y-axis tick marks!

In [3]:
longitude_xmax_ceil = math.ceil(longitude_xmax)
longitude_xmin_floor = math.floor(longitude_xmin)

latitude_ymax_ceil = math.ceil(latitude_ymax)
latitude_ymin_floor = math.floor(latitude_ymin)

print(longitude_xmax, longitude_xmax_ceil)
print(longitude_xmin, longitude_xmin_floor)
print(" ")
print(latitude_ymax, latitude_ymax_ceil)
print(latitude_ymin, latitude_ymin_floor)

-109.83 -109
-127.672 -128
 
42.461 43
28.938 28


In [4]:
iris_client = Client("IRIS", timeout=90000) #600 is reasonable timeout

In [5]:
cat = []
cat = iris_client.get_events(
                           minlat= latitude_ymin_floor, #ventura_minlat,
                           maxlat= latitude_ymax_ceil, #ventura_maxlat,
                           minlon= longitude_xmin_floor, #ventura_minlon,
                           maxlon= longitude_xmax_ceil, #ventura_maxlon,
                           starttime=UTCDateTime("1900-01-01"),
                           endtime=UTCDateTime("2023-11-01"),
                           minmag=0.0) # 2.0

In [6]:
times = []
lats = []
lons = []
deps = []
magnitudes = []
magnitudestype = []
for event in cat:
    if len(event.origins) != 0 and len(event.magnitudes) != 0:
        times.append(event.origins[0].time.datetime)
        lats.append(event.origins[0].latitude)
        lons.append(event.origins[0].longitude)
        deps.append(event.origins[0].depth)
        magnitudes.append(event.magnitudes[0].mag)
        magnitudestype.append(event.magnitudes[0].magnitude_type )

In [7]:
df = pd.DataFrame({'datetime':times, 'lat':lats,'lon':lons,'depth':deps,
                   'mag':magnitudes,'type':magnitudestype})

df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(by='datetime')
df = df.reset_index(drop=True)

print(df.shape)
print(df.columns)
display(df.head(4))
display(df.tail(4))

(562542, 6)
Index(['datetime', 'lat', 'lon', 'depth', 'mag', 'type'], dtype='object')


,datetime,lat,lon,depth,mag,type
0,1960-01-20 03:25:49,36.56,-121.71,0.0,5.0,None
1,1960-02-27 19:22:25,37.93,-122.02,NaN,3.4,None
2,1960-03-16 08:21:38,34.37,-118.93,NaN,3.2,None
3,1960-03-22 18:31:33,37.73,-122.13,NaN,2.6,None


,datetime,lat,lon,depth,mag,type
562538,2023-10-29 08:46:21.520,38.833668,-122.815834,1210.0,1.52,Md
562539,2023-10-29 09:01:19.990,38.687832,-122.501663,2420.0,1.69,Md
562540,2023-10-29 09:14:05.360,38.789665,-122.759499,330.0,1.36,Md
562541,2023-10-29 09:38:22.500,33.666833,-116.747333,12290.0,0.89,Ml


In [8]:
# df.to_csv('data_obspy_iris_AllCalBox_1960_to_2023sept24_20230924a.csv', index=False)

# df.to_csv('data_obspy_iris_AllCalBox_1960_to_2023oct29_20231029a.csv', index=False)